# SNP Dection Using Smith-Waterman Alignment Algorithm

In [1]:
import numpy as np
import time

start_time = time.time()

DELETION, INSERTION, MATCH = range(3)

def smith_waterman(sequence1, sequence2, mismatch=-1, insertion=-1, deletion=-1, match=2):
    seq1Length = len(sequence1)
    seq2Length = len(sequence2)
    score = np.zeros((seq1Length + 1, seq2Length + 1))
    score_record = np.zeros((seq1Length + 1, seq2Length + 1), dtype=int)

    for i in range(1, seq1Length + 1):
        for j in range(1, seq2Length + 1):
            deletion_score = ((score[i - 1][j] + deletion), DELETION)
            insertion_score = ((score[i][j - 1] + insertion), INSERTION)
            if sequence1[i - 1] == sequence2[j - 1]:
                match_score = ((score[i-1][j-1] + match), MATCH)
            else:
                match_score = ((score[i-1][j-1] + mismatch), MATCH)

            score[i][j], score_record[i][j] = max((0, 0), deletion_score, insertion_score, match_score)

    def traceback():
        max_score = 0
        max_pos = (0, 0)
        for i in range(1, seq1Length + 1):
            for j in range(1, seq2Length + 1):
                if score[i][j] > max_score:
                    max_score = score[i][j]
                    max_pos = (i, j)

        m, n = max_pos
        aligned_seq1 = []
        aligned_seq2 = []

        while score[m][n] != 0:
            direction = score_record[m][n] 
            if direction == MATCH: # moves diagonally
                m -= 1
                n -= 1
                aligned_seq1.append(sequence1[m])
                aligned_seq2.append(sequence2[n])
            elif direction == DELETION: # moves upward
                m -= 1
                aligned_seq1.append(sequence1[m])
                aligned_seq2.append('-')
            elif direction == INSERTION: # moves to the left
                n -= 1
                aligned_seq1.append('-')
                aligned_seq2.append(sequence2[n])
            else:
                break

        return [''.join(reversed(aligned_seq1)), ''.join(reversed(aligned_seq2))] # creating both aligned sequences

    return traceback()

def detect_snps(aligned1, aligned2):
    snps = []
    for i, (a, b) in enumerate(zip(aligned1, aligned2)):
        if a != b and a != '-' and b != '-':
            snps.append((i, a, b))  # (position, base_in_seq1, base_in_seq2)
    return snps

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time:.4f} seconds")

Execution time: 0.0006 seconds


In [10]:
import time
from Bio import SeqIO

fasta_file = "gene.fasta"
fasta_file1 = "gene1.fasta"

records = []
records1 = []
for i, record in enumerate(SeqIO.parse(fasta_file, "fasta")):
    records.append(record.seq[:2000]) # first 2000 base pairs
    break

for i, record in enumerate(SeqIO.parse(fasta_file1, "fasta")):
    records1.append(record.seq[:2000]) # first 2000 base pairs for alternate genome
    break
    
sequence1 = str(records[0])
sequence2 = str(records1[0])


print("First TP53 Sequence: " + sequence1)
print("Second TP53 Sequence: " + sequence2)

start_time = time.time()


aligned1, aligned2 = smith_waterman(sequence1, sequence2)
print("First Aligned Sequence: ", aligned1)
print("Second Aligned Sequence: ", aligned2)


snps = detect_snps(aligned1, aligned2)
print("SNPs found:")
for pos, base1, base2 in snps:
    print(f"Position {pos}: {base1} -> {base2}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time:.4f} seconds")

First Sequence: CTCAAAAGTCTAGAGCCACCGTCCAGGGAGCAGGTAGCTGCTGGGCTCCGGGGACACTTTGCGTTCGGGCTGGGAGCGTGCTTTCCACGACGGTGACACGCTTCCCTGGATTGGGTAAGCTCCTGACTGAACTTGATGAGTCCTCTCTGAGTCACGGGCTCTCGGCTCCGTGTATTTTCAGCTCGGGAAAATCGCTGGGGCTGGGGGTGGGGCAGTGGGGACTTAGCGAGTTTGGGGGTGAGTGGGATGGAAGCTTGGCTAGAGGGATCATCATAGGAGTTGCATTGTTGGGAGACCTGGGTGTAGATGATGGGGATGTTAGGACCATCCGAACTCAAAGTTGAACGCCTAGGCAGAGGAGTGGAGCTTTGGGGAACCTTGAGCCGGCCTAAAGCGTACTTCTTTGCACATCCACCCGGTGCTGGGCGTAGGGAATCCCTGAAATAAAAGATGCACAAAGCATTGAGGTCTGAGACTTTTGGATCTCGAAACATTGAGAACTCATAGCTGTATATTTTAGAGCCCATGGCATCCTAGTGAAAACTGGGGCTCCATTCCGAAATGATCATTTGGGGGTGATCCGGGGAGCCCAAGCTGCTAAGGTCCCACAACTTCCGGACCTTTGTCCTTCCTGGAGCGATCTTTCCAGGCAGCCCCCGGCTCCGCTAGATGGAGAAAATCCAATTGAAGGCTGTCAGTCGTGGAAGTGAGAAGTGCTAAACCAGGGGTTTGCCCGCCAGGCCGAGGAGGACCGTCGCAATCTGAGAGGCCCGGCAGCCCTGTTATTGTTTGGCTCCACATTTACATTTCTGCCTCTTGCAGCAGCATTTCCGGTTTCTTTTTGCCGGAGCAGCTCACTATTCACCCGATGAGAGGGGAGGAGAGAGAGAGAAAATGTCCTTTAGGCCGGTTCCTCTTACTTGGCAGAGGGAGGCTGCTATTCTCCGCCTGCATTTCTTTTTCTGGATTACTTAGTTATGGCCT